# **Naive Bayes Classifier in Python** <a class="anchor" id="0"></a>

El clasificador Naïve Bayes es un algoritmo sencillo y potente para la tarea de clasificación. 

Usaremos las herramietas de Scikit-Learn para implementar el algoritmo de clasificación Naive Bayes.

**Objetivo**: Clasificar noticias. Se le proporciona un dataset con noticias ya clasificadas en tres categorías: ocio, tecnología y cultura.

# Outline

Realizaremos éste modelo en tres pasos: 

1) Obtención de datos, filtrado y vectorización.

2) Implementar un transformador, para convertir a array los datos.

3) Crear el modelo de clasificador Naive Bayes en Python con distribución Gaussiana y guardarlo en formato Pickle para su posterior uso.

4) Teniendo ya nuestro modelo entrenado podremos hacer pruebas con cualquier noticia.

# Import libraries

In [ ]:
# Dependencies

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from sklearn.base import BaseEstimator
from scipy.sparse import issparse
from sklearn.naive_bayes import GaussianNB
# Un pipeline es un conjunto de tuberias o flujos que parten de una entrada (datos) 
# hacia una salida (modelo)
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle

#
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!pip install category_encoders

# Import dataset

In [ ]:
# Consiste en noticias clasificadas que nos servirán para entrenar nuestro modelo.

noticias = pd.read_csv("noticias.csv")

noticias.head()

In [ ]:
#

noticias.descripcion[0]

In [ ]:
#

noticias.categoria.value_counts()

In [ ]:
# Vamos a utilizar un listado de palabras que deseamos eliminar pues aportan poco o ningún 
# significado a la interpretación de las noticias. 
# Esas palabras, conocidas como stop-words.

# Palabras cuyo contenido no es muy relevante
with open("stopwords-es.json") as fname:
    stopwords_es = json.load(fname)
import nltk
#nltk.download('stopwordss')
#stopwords_es

In [ ]:
#
stopwords_es[10:23]

In [ ]:
# Debido a que trabajamos con texto, tenemos que vectorizar, usaremos Tf-idf
# Convert a collection of raw documents to a matrix of TF-IDF features.
vectorizador = TfidfVectorizer( stop_words = stopwords_es, max_features = 50000 )

# La transformamos en una matriz dispersa
vectorizador.fit_transform( noticias.descripcion )

In [ ]:
#

vectorizador

# Creamos una clase DenseTransformer, que nos permite convertir una matriz dispersa en un array

In [ ]:
#

class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

# Crear un pipeline que utilice los pasos anteriores, e incluya un clasificador Naive Bayes con distribución Gaussiana

En éste paso se crea un pipeline. 

Lo primero que recibe es el vectorizador que contiene una matriz dispersa que representa cada palabra asignada a cada noticia. 

Ésta matriz es pasada a DenseTransformer que la convierte a un array. 

Éste array es pasado a GaussianNB con lo que se crea un clasificador Naive Bayes. 

Se ajusta el modelo pasando como X (dato de entrada) la descripción de la noticia, y como y (dato de salida) la categoría. 

Utilizando pickle se guarda el modelo en un archivo que se puede utilizar más adelante. 

**Éste proceso puede tardar varios minutos.**

In [ ]:
# 

pipeline_gaussiano = make_pipeline(
    vectorizador,
    DenseTransformer(),
    GaussianNB()
)

In [ ]:
# Este puede tardar algunos minutos

pipeline_gaussiano.fit( X = noticias.descripcion, 
                        y = noticias.categoria )

In [ ]:
#
y_pred = pipeline_gaussiano.predict(noticias.descripcion)

y_pred

In [ ]:
#

accuracy_score(noticias.categoria, y_pred)

In [ ]:
#

cm = confusion_matrix(noticias.categoria, y_pred)

cm

In [ ]:
cm_matrix = pd.DataFrame( data = cm, 
                          columns = [ 'Actual Cultura', 'Actual Ocio', 'Actual Tecnología' ],
                          index = [ 'Predict Cultura', 'Predict Ocio', 'Predict Tecnología' ])

cm_matrix

In [ ]:
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame( data = cm, 
                          columns = [ 'Actual Cultura', 'Actual Ocio', 'Actual Tecnología' ],
                          index = [ 'Predict Cultura', 'Predict Ocio', 'Predict Tecnología' ])

sns.heatmap( cm_matrix, annot = True, fmt = 'd', cmap = 'YlGnBu')

plt.show()

In [ ]:
#

print(classification_report(noticias.categoria, y_pred))

In [ ]:
# Probemos el modelo
# https://www.bbc.com/mundo/articles/cxrkw471wnlo
Texto = [ 'El 29 de enero, Elon Musk publicaba en X el éxito de la primera intervención quirúrgica ' \
          'implantando un dispositivo desarrollado por su start up Neuralink en un humano. '\
          'El nombre del dispositivo: Telepathy (Telepatía).']

pipeline_gaussiano.predict(Texto)

In [ ]:
# Probemos el modelo
# https://www.eleconomista.com.mx/tecnologia/La-UE-abre-investigacion-contra-TikTok-por-violacion-a-normas-de-proteccion-a-menores-20240219-0034.html
Texto = [ 'La Unión Europea abrió un "procedimiento formal" contra la red social TikTok, '\
          'por posible violación de las normas en materia de protección a menores y transparencia, '\
         'anunció este lunes el comisario europeo de Mercado Interior, Thierry Breton.' ]

pipeline_gaussiano.predict(Texto)

In [ ]:
#

pickle.dump( pipeline_gaussiano, 
             open("naive_noticias_model.pickle", "wb") )

print ("Modelo Creado")

# Uso del modelo creado

Una vez creado nuestro modelo, podemos utilizarlo para clasificar una o miles de noticias.

Lo probamos cargando el modelo creado en archivo pickle y dandole una noticia. 

El resultado es la categoría en la que el clasificador pone a la noticia que introducimos.

In [ ]:
# Dependencies

import pickle
from sklearn.base import BaseEstimator
from scipy.sparse import issparse

In [ ]:
# Creamos una clase DenseTransformer, que nos permite convertir una matriz dispersa en un array

class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

In [ ]:
# Abrimos el modelo

pipeline_gaussiano = pickle.load(open("naive_noticias_model.pickle", "rb"))

pipeline_gaussiano

In [ ]:
#

n1 = [ "¿Cansado de que te roben Internet? Aqui puedes darte cuenta de quienes tienen tu wifi" ]
n2 = [ "Los cineastas mexicanos se unen para crear un fondo de emergencia ante la contingencia sanitaria" ]
n3 = [ "10 películas y series de terror en Netflix para ver este fin de semana" ]

r1 = pipeline_gaussiano.predict( n1 )
r2 = pipeline_gaussiano.predict( n2 )
r3 = pipeline_gaussiano.predict( n3 )

In [ ]:
#

print ("=====================")
print ("Noticia: " + str(n1))
print ("Categoría probable: " + str(r1))
print ("Noticia: " + str(n2))
print ("Categoría probable: " + str(r2))
print ("Noticia: " + str(n3))
print ("Categoría probable: " + str(r3))